<!-- TABS -->
# Create datatype

In [ ]:
from superduperdb import superduper

db = superduper("mongomock://test")

In [ ]:
# <tab: Vector>
from superduperdb import vector

datatype = vector(shape=(3, ))
origin_data = [0.1, 0.2, 0.3]

In [ ]:
# <tab: Tensor>
from superduperdb.ext.torch import tensor
import torch

datatype = tensor(torch.float, shape=(32, ))
origin_data = torch.randn(32)

In [ ]:
# <tab: Array>
from superduperdb.ext.numpy import array
import numpy as np

datatype = array(dtype="float64", shape=(32, ))
origin_data = np.random.randn(32)

In [ ]:
# <tab: Image>
from superduperdb.ext.pillow import pil_image
import PIL.Image

!wget https://raw.githubusercontent.com/openai/CLIP/main/CLIP.png

datatype = pil_image
origin_data = PIL.Image.open("CLIP.png")

In [ ]:
# <tab: Audio>
!wget https://ccrma.stanford.edu/workshops/mir2014/audio/simpleLoop.wav

import librosa
from superduperdb.ext.numpy import array
import numpy as np
from IPython.display import Audio

datatype = array(dtype="float32", shape=(None, ))
origin_data, fs = librosa.load('simpleLoop.wav')

Audio(origin_data, rate=fs)

In [ ]:
# <tab: Video>
from superduperdb import DataType

# Create an instance of the Encoder with the identifier 'video_on_file' and load_hybrid set to False
datatype = DataType(
    identifier='video_on_file',
    encodable='artifact',
)

In [ ]:
# <tab: Custom-in-DB>
...

In [ ]:
# <tab: Custom-Artifact>
...

## Interact with the database.

In [ ]:
db.add(datatype)

### Insert data into the database

In [ ]:
from superduperdb.backends.mongodb import Collection
from superduperdb import Document
collection = Collection("data")

In [ ]:
db.execute(collection.insert_one(Document({"x": datatype(origin_data)})))

### Read data from the database

In [ ]:
data = db.execute(collection.find_one())
data

**Restore data to its original form**

In [ ]:
data.unpack()

In [ ]:
data.unpack()["x"]